# 🚀 Projeto Final 2: Sistema RAG Avançado com Agents - O "Mega-Cérebro" da IA

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_01.png)

Fala galera! 👋 É o Pedro aqui e tá na hora do nosso **segundo projeto final**! 

Tá, mas o que vamos fazer hoje? Bora criar um sistema **COMPLETO** que junta **TUDO** que aprendemos até agora:
- RAG (nosso queridinho dos documentos)
- Agents (os espertinhos que fazem decisões)
- Memory (para lembrar das conversas)
- Tools personalizadas
- E muito mais!

É como se fosse um "mega-cérebro" da IA que pode:
1. 📚 Pesquisar em documentos
2. 🧮 Fazer cálculos
3. 🌐 Buscar na internet
4. 💭 Lembrar de conversas anteriores
5. 🎯 Tomar decisões inteligentes

**Dica do Pedro**: Este projeto vai servir de base para o próximo módulo onde vamos fazer o deploy no Streamlit!

## 🎯 Objetivos do Projeto

Nosso sistema vai ser como um **assistente pessoal turbinado** que pode:

### 📋 Funcionalidades Principais:
- **RAG Inteligente**: Buscar informações em múltiplos documentos
- **Agent com Tools**: Escolher a ferramenta certa para cada tarefa
- **Memória Persistente**: Lembrar do contexto da conversa
- **Análise de Dados**: Processar e analisar informações
- **Busca Web**: Quando não souber algo, vai atrás!

### 🏗️ Arquitetura do Sistema:

```mermaid
graph TD
    A[Usuário] --> B[Agent Controller]
    B --> C[RAG Tool]
    B --> D[Calculator Tool]
    B --> E[Web Search Tool]
    B --> F[Data Analysis Tool]
    C --> G[Vector Store]
    G --> H[Documentos]
    B --> I[Memory System]
    I --> J[Resposta Final]
    J --> A
```

**Analogia do Pedro**: É como ter um assistente que tem acesso a uma biblioteca gigante (RAG), uma calculadora (tools), Google (web search) e uma memória de elefante (memory system)!

In [ ]:
# Instalação dos pacotes necessários - Nosso kit de ferramentas completo!
!pip install -q langchain langchain-community langchain-google-genai
!pip install -q chromadb sentence-transformers
!pip install -q wikipedia-api requests beautifulsoup4
!pip install -q pandas matplotlib seaborn
!pip install -q python-dotenv

In [ ]:
# Imports fundamentais - Nosso arsenal completo!
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# LangChain Core
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

# RAG Components
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.schema import Document

# Web e Utilitários
import requests
from bs4 import BeautifulSoup

print("✅ Todos os imports carregados! Bora para a ação!")

In [ ]:
# Configuração da API - A chave do reino!
from google.colab import userdata

# Pegando nossa API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Inicializando nossos modelos principais
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.1,
    max_tokens=2048
)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

print("🔑 Modelos configurados! O Gemini está pronto para a batalha!")

## 🏗️ Construindo o Sistema RAG Avançado

Agora vamos construir nosso sistema RAG, mas não um RAG qualquer... Um **RAG TURBINADO**! 🚀

### 📚 O que é um RAG Avançado?

Lembra do RAG básico que fizemos no módulo 8? Pois é, agora vamos evoluir ele! Um RAG avançado tem:

- **Múltiplas fontes de dados**: Não só um documento, mas vários!
- **Busca inteligente**: Usa diferentes estratégias de busca
- **Filtragem de resultados**: Só pega o que é realmente relevante
- **Fallback para web**: Se não achar nos docs, busca na internet!

**Dica do Pedro**: É como ter uma biblioteca que, se não tem o livro que você quer, automaticamente pede emprestado de outras bibliotecas!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_02.png)

In [ ]:
# Vamos criar alguns documentos de exemplo para nosso RAG
# Na vida real, você carregaria documentos reais!

sample_documents = [
    {
        "content": """
        Inteligência Artificial e Machine Learning
        
        A Inteligência Artificial (IA) é uma área da ciência da computação que se concentra 
        no desenvolvimento de sistemas capazes de realizar tarefas que normalmente requerem 
        inteligência humana. Machine Learning é uma subárea da IA que permite que os 
        computadores aprendam e melhorem automaticamente através da experiência.
        
        Os principais tipos de ML incluem:
        - Aprendizado Supervisionado: Usa dados rotulados
        - Aprendizado Não Supervisionado: Encontra padrões em dados não rotulados  
        - Aprendizado por Reforço: Aprende através de recompensas e punições
        """,
        "metadata": {"source": "ml_guide.txt", "category": "technology"}
    },
    {
        "content": """
        LangChain Framework
        
        LangChain é um framework poderoso para desenvolvimento de aplicações com 
        Large Language Models (LLMs). Ele fornece uma série de componentes modulares 
        que podem ser combinados para criar aplicações complexas.
        
        Componentes principais:
        - Models: Interface para diferentes LLMs
        - Prompts: Templates para formatação de entradas
        - Chains: Sequências de operações
        - Agents: Sistemas que podem usar ferramentas
        - Memory: Sistemas de memória para conversas
        - Retrievers: Para busca em documentos (RAG)
        """,
        "metadata": {"source": "langchain_intro.txt", "category": "technology"}
    },
    {
        "content": """
        Análise de Dados com Python
        
        Python é uma das linguagens mais populares para análise de dados, 
        oferecendo bibliotecas poderosas como Pandas, NumPy e Matplotlib.
        
        Principais bibliotecas:
        - Pandas: Manipulação e análise de dados estruturados
        - NumPy: Computação numérica eficiente
        - Matplotlib/Seaborn: Visualização de dados
        - Scikit-learn: Machine Learning
        - Jupyter: Ambiente interativo para desenvolvimento
        
        Processo típico de análise:
        1. Coleta e limpeza dos dados
        2. Exploração e visualização
        3. Modelagem e análise
        4. Interpretação dos resultados
        """,
        "metadata": {"source": "python_analysis.txt", "category": "data_science"}
    }
]

# Convertendo para objetos Document do LangChain
documents = []
for doc in sample_documents:
    documents.append(Document(
        page_content=doc["content"],
        metadata=doc["metadata"]
    ))

print(f"📚 Criados {len(documents)} documentos de exemplo!")
print("Categorias:", [doc.metadata['category'] for doc in documents])

In [ ]:
# Criando nosso RAG System - O cérebro da operação!
class AdvancedRAGSystem:
    def __init__(self, documents, embeddings, llm):
        self.llm = llm
        self.embeddings = embeddings
        
        # Splitter para quebrar documentos grandes
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        
        # Processando documentos
        self.docs = self.text_splitter.split_documents(documents)
        
        # Criando o vector store
        self.vector_store = Chroma.from_documents(
            documents=self.docs,
            embedding=self.embeddings,
            collection_name="advanced_rag"
        )
        
        # Retriever com configurações avançadas
        self.retriever = self.vector_store.as_retriever(
            search_type="mmr",  # Maximum Marginal Relevance
            search_kwargs={"k": 3, "fetch_k": 6}
        )
        
        print("🏗️ RAG System construído com sucesso!")
        print(f"📄 {len(self.docs)} chunks processados")
    
    def search_documents(self, query: str) -> str:
        """Busca documentos relevantes para a query"""
        try:
            # Buscando documentos relevantes
            relevant_docs = self.retriever.get_relevant_documents(query)
            
            if not relevant_docs:
                return "Nenhum documento relevante encontrado."
            
            # Formatando os resultados
            context = "\n\n--- DOCUMENTOS ENCONTRADOS ---\n\n"
            for i, doc in enumerate(relevant_docs, 1):
                context += f"Documento {i} (Fonte: {doc.metadata.get('source', 'desconhecida')}):\n"
                context += doc.page_content + "\n\n"
            
            # Gerando resposta com contexto
            prompt = f"""
            Com base nos documentos fornecidos, responda à seguinte pergunta: {query}
            
            Contexto dos documentos:
            {context}
            
            Resposta (seja preciso e cite as fontes quando relevante):
            """
            
            response = self.llm.invoke(prompt)
            return response.content
            
        except Exception as e:
            return f"Erro na busca: {str(e)}"

# Inicializando nosso RAG System
rag_system = AdvancedRAGSystem(documents, embeddings, llm)

# Testando o sistema
test_query = "O que é LangChain e quais são seus principais componentes?"
result = rag_system.search_documents(test_query)

print("\n🧪 TESTE DO RAG SYSTEM:")
print(f"Pergunta: {test_query}")
print(f"\nResposta:\n{result}")

## 🔧 Criando Tools Personalizadas

Agora vamos criar as **ferramentas** que nosso Agent vai usar! Lembra do módulo 9? Vamos turbinar aquelas ideias!

### 🛠️ Por que Tools Personalizadas?

As tools são como **superpoderes** para nosso Agent:
- Cada tool resolve um tipo específico de problema
- O Agent escolhe qual tool usar baseado na pergunta
- Podemos ter quantas tools quisermos!

**Analogia do Pedro**: É como um canivete suíço! Cada ferramenta tem sua função específica, e o Agent é esperto o suficiente para escolher a certa!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_03.png)

In [ ]:
# Tool 1: Calculator - Para cálculos matemáticos
def calculator_tool(expression: str) -> str:
    """
    Realiza cálculos matemáticos seguros.
    Use para operações como: 2+2, 10*5, sqrt(16), etc.
    """
    try:
        # Limpando a expressão e permitindo funções matemáticas seguras
        import math
        
        # Dicionário de funções permitidas
        allowed_functions = {
            'sqrt': math.sqrt,
            'sin': math.sin,
            'cos': math.cos,
            'tan': math.tan,
            'log': math.log,
            'exp': math.exp,
            'pi': math.pi,
            'e': math.e
        }
        
        # Avaliando a expressão de forma segura
        result = eval(expression, {"__builtins__": {}}, allowed_functions)
        return f"Resultado: {result}"
        
    except Exception as e:
        return f"Erro no cálculo: {str(e)}. Use expressões matemáticas válidas como: 2+2, sqrt(16), etc."

# Tool 2: Web Search - Para buscar na internet
def web_search_tool(query: str) -> str:
    """
    Busca informações na internet quando não há dados suficientes nos documentos.
    Use para informações atuais, notícias, ou dados não disponíveis localmente.
    """
    try:
        # Simulando uma busca web (na vida real, use APIs como Google Custom Search)
        # Por questões de simplicidade, vamos simular alguns resultados
        
        simulated_results = {
            "clima": "O clima hoje está ensolarado com temperatura de 25°C.",
            "notícias": "Últimas notícias: Avanços em IA continuam crescendo exponencialmente.",
            "bitcoin": "Bitcoin está cotado em aproximadamente $43,000 USD hoje.",
            "python": "Python continua sendo uma das linguagens mais populares para ciência de dados."
        }
        
        # Busca por palavras-chave
        query_lower = query.lower()
        for key, result in simulated_results.items():
            if key in query_lower:
                return f"Resultado da busca web: {result}"
        
        return f"Busca web para '{query}': Informações não encontradas na simulação. Na implementação real, seria usado uma API de busca."
        
    except Exception as e:
        return f"Erro na busca web: {str(e)}"

# Tool 3: Data Analysis - Para análise de dados
def data_analysis_tool(data_request: str) -> str:
    """
    Realiza análise de dados simples e gera visualizações.
    Use para requests como: 'criar gráfico', 'analisar dados', 'estatísticas'.
    """
    try:
        # Criando dados de exemplo para demonstração
        np.random.seed(42)
        data = {
            'vendas': np.random.normal(1000, 200, 30),
            'lucro': np.random.normal(150, 50, 30),
            'clientes': np.random.randint(50, 200, 30)
        }
        
        df = pd.DataFrame(data)
        
        # Análise básica
        stats = df.describe()
        
        analysis = f"""
        ANÁLISE DE DADOS GERADOS:
        
        Estatísticas Descritivas:
        - Vendas médias: R$ {df['vendas'].mean():.2f}
        - Lucro médio: R$ {df['lucro'].mean():.2f}
        - Clientes médios: {df['clientes'].mean():.0f}
        
        Correlações:
        - Vendas vs Lucro: {df['vendas'].corr(df['lucro']):.3f}
        - Vendas vs Clientes: {df['vendas'].corr(df['clientes']):.3f}
        
        Insights: Os dados mostram uma correlação moderada entre vendas e lucro.
        """
        
        return analysis
        
    except Exception as e:
        return f"Erro na análise: {str(e)}"

print("🔧 Tools criadas com sucesso!")
print("✅ Calculator Tool - Para cálculos")
print("✅ Web Search Tool - Para busca na internet")
print("✅ Data Analysis Tool - Para análise de dados")

In [ ]:
# Criando as Tools do LangChain
tools = [
    Tool(
        name="RAG_Search",
        func=rag_system.search_documents,
        description="Busca informações em documentos sobre IA, LangChain, Python e análise de dados. Use quando a pergunta for sobre estes tópicos."
    ),
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="Realiza cálculos matemáticos. Use para operações como soma, subtração, multiplicação, divisão, raiz quadrada, etc. Exemplo: '2+2' ou 'sqrt(16)'"
    ),
    Tool(
        name="Web_Search",
        func=web_search_tool,
        description="Busca informações na internet quando não há dados suficientes nos documentos locais. Use para informações atuais ou específicas."
    ),
    Tool(
        name="Data_Analysis",
        func=data_analysis_tool,
        description="Realiza análise de dados e gera estatísticas. Use quando precisar analisar dados ou gerar insights."
    )
]

print("🛠️ LangChain Tools configuradas!")
for tool in tools:
    print(f"- {tool.name}: {tool.description[:50]}...")

## 🧠 Sistema de Memória Avançado

Agora vamos implementar um sistema de memória mais sofisticado! Lembra do módulo 5? Vamos evoluir aqueles conceitos!

### 🐘 Por que Memória é Crucial?

A memória permite que nosso Agent:
- **Mantenha contexto** entre perguntas
- **Aprenda** com interações anteriores
- **Seja mais natural** nas conversas
- **Evite repetições** desnecessárias

### 📊 Tipos de Memória que vamos usar:

$$ \text{Memória Total} = \text{Conversa} + \text{Contexto} + \text{Preferências} $$

**Dica do Pedro**: É como a diferença entre falar com alguém que te conhece há anos versus um estranho. A memória faz toda a diferença!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_04.png)

In [ ]:
# Sistema de Memória Avançado
class AdvancedMemorySystem:
    def __init__(self, window_size=10):
        # Memória principal para conversas
        self.conversation_memory = ConversationBufferWindowMemory(
            k=window_size,
            memory_key="chat_history",
            return_messages=True
        )
        
        # Memória de longo prazo para preferências do usuário
        self.user_preferences = {}
        
        # Cache de consultas frequentes
        self.query_cache = {}
        
        # Estatísticas de uso
        self.usage_stats = {
            'total_queries': 0,
            'tool_usage': {},
            'common_topics': {}
        }
        
        print("🧠 Sistema de Memória Avançado inicializado!")
    
    def add_user_preference(self, key: str, value: str):
        """Adiciona uma preferência do usuário"""
        self.user_preferences[key] = value
        print(f"✅ Preferência salva: {key} = {value}")
    
    def get_user_context(self) -> str:
        """Retorna o contexto do usuário para personalização"""
        if not self.user_preferences:
            return "Novo usuário, sem preferências salvas."
        
        context = "Preferências do usuário: "
        for key, value in self.user_preferences.items():
            context += f"{key}: {value}; "
        
        return context
    
    def update_stats(self, query: str, tool_used: str = None):
        """Atualiza estatísticas de uso"""
        self.usage_stats['total_queries'] += 1
        
        if tool_used:
            if tool_used not in self.usage_stats['tool_usage']:
                self.usage_stats['tool_usage'][tool_used] = 0
            self.usage_stats['tool_usage'][tool_used] += 1
        
        # Análise simples de tópicos
        query_lower = query.lower()
        topics = ['ia', 'python', 'dados', 'cálculo', 'langchain']
        
        for topic in topics:
            if topic in query_lower:
                if topic not in self.usage_stats['common_topics']:
                    self.usage_stats['common_topics'][topic] = 0
                self.usage_stats['common_topics'][topic] += 1
    
    def get_stats_summary(self) -> str:
        """Retorna um resumo das estatísticas"""
        stats = self.usage_stats
        
        summary = f"""
        📊 ESTATÍSTICAS DE USO:
        
        Total de consultas: {stats['total_queries']}
        
        Tools mais usadas:
        {dict(sorted(stats['tool_usage'].items(), key=lambda x: x[1], reverse=True))}
        
        Tópicos mais consultados:
        {dict(sorted(stats['common_topics'].items(), key=lambda x: x[1], reverse=True))}
        
        Preferências do usuário: {len(self.user_preferences)} salvas
        """
        
        return summary

# Inicializando o sistema de memória
memory_system = AdvancedMemorySystem(window_size=15)

# Testando com algumas preferências
memory_system.add_user_preference("linguagem_preferida", "Python")
memory_system.add_user_preference("nivel_tecnico", "Avançado")
memory_system.add_user_preference("area_interesse", "IA e Machine Learning")

print("\n📋 Contexto do usuário:")
print(memory_system.get_user_context())

## 🤖 Criando o Agent Master - O Cérebro Central

Agora chegou a hora do **show**! Vamos criar nosso Agent principal que vai orquestrar tudo!

### 🎯 O que é um Agent Master?

É como um **maestro de orquestra** que:
- 🎼 **Coordena** todas as tools
- 🧠 **Decide** qual ferramenta usar
- 💭 **Mantém** o contexto da conversa
- 🎯 **Entrega** respostas precisas

### 🔄 Fluxo de Decisão do Agent:

```mermaid
graph TD
    A[Pergunta do Usuário] --> B[Agent Analisa]
    B --> C{Tipo de Pergunta?}
    C -->|Conhecimento| D[RAG Search]
    C -->|Cálculo| E[Calculator]
    C -->|Web Info| F[Web Search]
    C -->|Dados| G[Data Analysis]
    D --> H[Resposta Final]
    E --> H
    F --> H
    G --> H
    H --> I[Atualiza Memória]
```

**Dica do Pedro**: O Agent usa ReAct (Reasoning + Acting), que significa que ele pensa antes de agir!

In [ ]:
# Criando o Prompt Template para nosso Agent
agent_prompt = PromptTemplate(
    input_variables=["tools", "tool_names", "input", "agent_scratchpad", "chat_history", "user_context"],
    template="""
Você é um assistente IA avançado com acesso a múltiplas ferramentas especializadas.

CONTEXTO DO USUÁRIO: {user_context}

HISTÓRICO DA CONVERSA:
{chat_history}

FERRAMENTAS DISPONÍVEIS:
{tools}

Use o seguinte formato para responder:

Question: a pergunta que você deve responder
Thought: você deve sempre pensar sobre o que fazer
Action: a ação a tomar, deve ser uma de [{tool_names}]
Action Input: a entrada para a ação
Observation: o resultado da ação
... (esse Thought/Action/Action Input/Observation pode repetir N vezes)
Thought: Eu sei a resposta final
Final Answer: a resposta final para a pergunta original

REGRAS IMPORTANTES:
1. Use RAG_Search para perguntas sobre IA, LangChain, Python, análise de dados
2. Use Calculator para operações matemáticas
3. Use Web_Search para informações atuais ou não disponíveis localmente
4. Use Data_Analysis para análise de dados e estatísticas
5. Seja sempre preciso e cite fontes quando relevante
6. Mantenha o contexto da conversa anterior
7. Responda em português de forma clara e didática

Question: {input}
Thought:{agent_scratchpad}
"""
)

print("📝 Prompt do Agent configurado!")
print("🎯 O Agent agora sabe exatamente como usar cada ferramenta!")

In [ ]:
# Criando o Agent Master - O cérebro da operação!
class AgentMaster:
    def __init__(self, llm, tools, memory_system, agent_prompt):
        self.llm = llm
        self.tools = tools
        self.memory_system = memory_system
        
        # Criando o agent ReAct
        self.agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=agent_prompt
        )
        
        # Executor do agent com memória
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory_system.conversation_memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3,
            max_execution_time=60
        )
        
        print("🤖 Agent Master criado com sucesso!")
        print("🧠 ReAct Agent configurado com memória")
        print(f"🛠️ {len(self.tools)} ferramentas disponíveis")
    
    def chat(self, message: str) -> str:
        """Interface principal para conversar com o agent"""
        try:
            # Preparando o contexto do usuário
            user_context = self.memory_system.get_user_context()
            
            # Executando o agent
            response = self.agent_executor.invoke({
                "input": message,
                "user_context": user_context
            })
            
            # Atualizando estatísticas
            self.memory_system.update_stats(message)
            
            return response["output"]
            
        except Exception as e:
            error_msg = f"Ops! Ocorreu um erro: {str(e)}"
            print(f"❌ {error_msg}")
            return error_msg
    
    def get_system_status(self) -> str:
        """Retorna o status do sistema"""
        status = f"""
        🤖 STATUS DO AGENT MASTER:
        
        ✅ Agent ReAct: Ativo
        ✅ Ferramentas: {len(self.tools)} disponíveis
        ✅ Memória: Ativa (janela de {self.memory_system.conversation_memory.k} mensagens)
        ✅ LLM: {type(self.llm).__name__}
        
        {self.memory_system.get_stats_summary()}
        """
        return status

# Inicializando nosso Agent Master
agent_master = AgentMaster(
    llm=llm,
    tools=tools,
    memory_system=memory_system,
    agent_prompt=agent_prompt
)

print("\n🎉 SISTEMA COMPLETO INICIALIZADO!")
print(agent_master.get_system_status())

## 🧪 Testando Nosso Sistema Completo

Agora é a hora da verdade! Vamos testar nosso **mega-sistema** com diferentes tipos de perguntas!

### 🎯 Cenários de Teste:
1. **RAG**: Perguntas sobre nossos documentos
2. **Cálculos**: Operações matemáticas
3. **Busca Web**: Informações externas  
4. **Análise**: Processamento de dados
5. **Memória**: Continuidade da conversa

**Dica do Pedro**: Observe como o Agent **pensa** antes de agir! Isso é o ReAct em ação!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_05.png)

In [ ]:
# Teste 1: RAG - Pergunta sobre nossos documentos
print("🧪 TESTE 1: RAG SEARCH")
print("=" * 50)

question1 = "Quais são os principais tipos de Machine Learning?"
print(f"Pergunta: {question1}")
print("\nResposta:")

response1 = agent_master.chat(question1)
print(response1)
print("\n" + "="*50)

In [ ]:
# Teste 2: Calculator - Operação matemática
print("🧪 TESTE 2: CALCULATOR")
print("=" * 50)

question2 = "Quanto é a raiz quadrada de 144 multiplicada por 5?"
print(f"Pergunta: {question2}")
print("\nResposta:")

response2 = agent_master.chat(question2)
print(response2)
print("\n" + "="*50)

In [ ]:
# Teste 3: Análise de Dados
print("🧪 TESTE 3: DATA ANALYSIS")
print("=" * 50)

question3 = "Pode fazer uma análise de dados para mim?"
print(f"Pergunta: {question3}")
print("\nResposta:")

response3 = agent_master.chat(question3)
print(response3)
print("\n" + "="*50)

In [ ]:
# Teste 4: Teste de Memória - Pergunta que referencia conversa anterior
print("🧪 TESTE 4: MEMÓRIA")
print("=" * 50)

question4 = "Baseado no que conversamos sobre Machine Learning, qual tipo seria melhor para classificação de imagens?"
print(f"Pergunta: {question4}")
print("\nResposta:")

response4 = agent_master.chat(question4)
print(response4)
print("\n" + "="*50)

In [ ]:
# Teste 5: Web Search
print("🧪 TESTE 5: WEB SEARCH")
print("=" * 50)

question5 = "Qual é o preço atual do Bitcoin?"
print(f"Pergunta: {question5}")
print("\nResposta:")

response5 = agent_master.chat(question5)
print(response5)
print("\n" + "="*50)

In [ ]:
# Visualizando o status final do sistema
print("📊 STATUS FINAL DO SISTEMA")
print(agent_master.get_system_status())

# Criando um gráfico das estatísticas de uso
stats = memory_system.usage_stats

if stats['tool_usage']:
    plt.figure(figsize=(12, 5))
    
    # Gráfico 1: Uso das Tools
    plt.subplot(1, 2, 1)
    tools_names = list(stats['tool_usage'].keys())
    tools_counts = list(stats['tool_usage'].values())
    
    plt.bar(tools_names, tools_counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    plt.title('Uso das Ferramentas', fontsize=14, fontweight='bold')
    plt.xlabel('Ferramentas')
    plt.ylabel('Número de Usos')
    plt.xticks(rotation=45)
    
    # Gráfico 2: Tópicos mais consultados
    plt.subplot(1, 2, 2)
    if stats['common_topics']:
        topics = list(stats['common_topics'].keys())
        counts = list(stats['common_topics'].values())
        
        plt.pie(counts, labels=topics, autopct='%1.1f%%', startangle=90,
               colors=['#FFB6C1', '#87CEEB', '#DDA0DD', '#98FB98', '#F0E68C'])
        plt.title('Tópicos Mais Consultados', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("📈 Gráficos de uso gerados!")
else:
    print("📊 Ainda não há dados suficientes para gráficos")

## 🎯 Exercício Prático: Teste o Sistema!

Agora é a sua vez de testar nosso sistema! 💪

### 📝 Desafio:
Faça pelo menos **3 perguntas diferentes** que testem diferentes funcionalidades:

1. **Uma pergunta sobre nossos documentos** (para testar RAG)
2. **Um cálculo matemático complexo** (para testar Calculator)
3. **Uma pergunta que combine múltiplas tools**

### 🎁 Exemplos de perguntas interessantes:
- "O que é LangChain e qual seria o resultado de sqrt(25) * 10?"
- "Baseado nos documentos, explique IA e depois faça uma análise de dados"
- "Compare Python com outras linguagens e calcule 2^8"

**Dica do Pedro**: Observe como o Agent **raciocina** e escolhe as ferramentas certas!

In [ ]:
# 🎯 SEU TESTE AQUI!
# Substitua a pergunta abaixo pela sua própria pergunta

# Teste 1 - Sua pergunta sobre documentos
sua_pergunta_1 = "[SUBSTITUA AQUI] - Ex: Explique o que são Chains no LangChain"

# Descomente e teste:
# print("🎯 SEU TESTE 1:")
# print(f"Pergunta: {sua_pergunta_1}")
# resposta1 = agent_master.chat(sua_pergunta_1)
# print(f"Resposta: {resposta1}")

print("✏️ Modifique as perguntas acima e teste o sistema!")
print("💡 Dica: Tente perguntas que combinem diferentes ferramentas!")

In [ ]:
# 🎯 TESTE COMBINADO - Exemplo de pergunta complexa
pergunta_complexa = "Explique o que é LangChain, calcule quantos componentes principais ele tem multiplicado por 3, e depois faça uma análise dos dados que você possui."

print("🎯 TESTE COMPLEXO - Múltiplas Tools:")
print(f"Pergunta: {pergunta_complexa}")
print("\nProcessando... (pode demorar um pouco)")

resposta_complexa = agent_master.chat(pergunta_complexa)
print(f"\nResposta Completa:")
print(resposta_complexa)

print("\n🎉 Liiindo! Viu como o Agent conseguiu usar múltiplas ferramentas?")

## 🚀 Melhorias e Otimizações

Nosso sistema está funcionando, mas sempre podemos melhorar! Vamos ver algumas **otimizações avançadas**:

### 🔧 Possíveis Melhorias:

1. **Cache Inteligente**: Salvar respostas frequentes
2. **Análise de Sentimento**: Entender o humor do usuário
3. **Auto-avaliação**: O Agent se auto-criticar
4. **Streaming**: Respostas em tempo real
5. **Multi-modal**: Processar imagens e áudio

### 📈 Métricas de Performance:

$$ \text{Eficiência} = \frac{\text{Respostas Corretas}}{\text{Total de Tentativas}} \times \frac{1}{\text{Tempo Médio}} $$

**Dica do Pedro**: Em produção, sempre monitore performance e precisão!

In [ ]:
# Sistema de Cache Simples
class CacheSystem:
    def __init__(self, max_size=100):
        self.cache = {}
        self.max_size = max_size
        self.access_count = {}
    
    def get(self, key: str):
        """Busca no cache"""
        if key in self.cache:
            self.access_count[key] = self.access_count.get(key, 0) + 1
            return self.cache[key]
        return None
    
    def set(self, key: str, value: str):
        """Salva no cache"""
        if len(self.cache) >= self.max_size:
            # Remove o menos usado
            least_used = min(self.access_count, key=self.access_count.get)
            del self.cache[least_used]
            del self.access_count[least_used]
        
        self.cache[key] = value
        self.access_count[key] = 1
    
    def get_stats(self):
        """Estatísticas do cache"""
        return {
            'size': len(self.cache),
            'max_size': self.max_size,
            'hit_rate': sum(self.access_count.values()),
            'most_accessed': max(self.access_count, key=self.access_count.get) if self.access_count else None
        }

# Sistema de Métricas
class PerformanceMonitor:
    def __init__(self):
        self.response_times = []
        self.success_count = 0
        self.error_count = 0
        
    def log_response(self, response_time: float, success: bool):
        """Registra uma resposta"""
        self.response_times.append(response_time)
        if success:
            self.success_count += 1
        else:
            self.error_count += 1
    
    def get_metrics(self):
        """Calcula métricas de performance"""
        if not self.response_times:
            return "Nenhuma métrica disponível ainda."
        
        avg_time = np.mean(self.response_times)
        success_rate = self.success_count / (self.success_count + self.error_count) * 100
        
        return f"""
        📊 MÉTRICAS DE PERFORMANCE:
        
        ⏱️ Tempo médio de resposta: {avg_time:.2f}s
        ✅ Taxa de sucesso: {success_rate:.1f}%
        📈 Total de consultas: {len(self.response_times)}
        ❌ Erros: {self.error_count}
        """

# Inicializando sistemas de otimização
cache_system = CacheSystem()
performance_monitor = PerformanceMonitor()

print("🔧 Sistemas de otimização inicializados!")
print("✅ Cache System: Ativo")
print("✅ Performance Monitor: Ativo")

## 📊 Visualização Final - Dashboard do Sistema

Vamos criar um **dashboard visual** para monitorar nosso sistema!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_06.png)

In [ ]:
# Dashboard Visual do Sistema
def create_system_dashboard():
    """Cria um dashboard completo do sistema"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('🤖 Dashboard do Agent Master System', fontsize=16, fontweight='bold')
    
    # Gráfico 1: Uso das Tools
    ax1 = axes[0, 0]
    if memory_system.usage_stats['tool_usage']:
        tools = list(memory_system.usage_stats['tool_usage'].keys())
        counts = list(memory_system.usage_stats['tool_usage'].values())
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
        
        bars = ax1.bar(tools, counts, color=colors[:len(tools)])
        ax1.set_title('Uso das Ferramentas', fontweight='bold')
        ax1.set_ylabel('Número de Usos')
        
        # Adicionando valores nas barras
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom')
    else:
        ax1.text(0.5, 0.5, 'Nenhum dado\ndisponível', ha='center', va='center',
                transform=ax1.transAxes, fontsize=12)
        ax1.set_title('Uso das Ferramentas', fontweight='bold')
    
    # Gráfico 2: Simulação de Performance ao longo do tempo
    ax2 = axes[0, 1]
    time_points = np.arange(1, 11)
    performance = np.random.normal(0.85, 0.1, 10)
    performance = np.clip(performance, 0.6, 1.0)  # Mantém entre 60% e 100%
    
    ax2.plot(time_points, performance, marker='o', linewidth=2, color='#45B7D1')
    ax2.fill_between(time_points, performance, alpha=0.3, color='#45B7D1')
    ax2.set_title('Performance ao Longo do Tempo', fontweight='bold')
    ax2.set_xlabel('Interações')
    ax2.set_ylabel('Taxa de Sucesso')
    ax2.set_ylim(0.5, 1.0)
    ax2.grid(True, alpha=0.3)
    
    # Gráfico 3: Distribuição de Tópicos
    ax3 = axes[1, 0]
    if memory_system.usage_stats['common_topics']:
        topics = list(memory_system.usage_stats['common_topics'].keys())
        topic_counts = list(memory_system.usage_stats['common_topics'].values())
        colors = ['#FFB6C1', '#87CEEB', '#DDA0DD', '#98FB98', '#F0E68C']
        
        wedges, texts, autotexts = ax3.pie(topic_counts, labels=topics, autopct='%1.1f%%',
                                          colors=colors[:len(topics)], startangle=90)
        ax3.set_title('Distribuição de Tópicos', fontweight='bold')
    else:
        ax3.text(0.5, 0.5, 'Nenhum tópico\nidentificado', ha='center', va='center',
                transform=ax3.transAxes, fontsize=12)
        ax3.set_title('Distribuição de Tópicos', fontweight='bold')
    
    # Gráfico 4: Métricas do Sistema
    ax4 = axes[1, 1]
    metrics_names = ['Precisão', 'Velocidade', 'Memória', 'Satisfação']
    metrics_values = [0.89, 0.75, 0.92, 0.87]  # Valores simulados
    
    y_pos = np.arange(len(metrics_names))
    bars = ax4.barh(y_pos, metrics_values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels(metrics_names)
    ax4.set_xlabel('Score (0-1)')
    ax4.set_title('Métricas do Sistema', fontweight='bold')
    ax4.set_xlim(0, 1)
    
    # Adicionando valores nas barras
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax4.text(width + 0.01, bar.get_y() + bar.get_height()/2.,
                f'{metrics_values[i]:.2f}', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()

# Gerando o dashboard
print("📊 Gerando Dashboard do Sistema...")
create_system_dashboard()
print("✅ Dashboard criado com sucesso!")

## 🎓 Resumo e Próximos Passos

**Parabéns!** 🎉 Você acabou de construir um sistema de IA **COMPLETO** e **AVANÇADO**!

### 🏆 O que Conseguimos:

✅ **RAG Avançado**: Sistema de busca inteligente em documentos  
✅ **Agent ReAct**: IA que pensa antes de agir  
✅ **Multiple Tools**: Calculadora, busca web, análise de dados  
✅ **Memória Persistente**: Contexto entre conversas  
✅ **Sistema de Cache**: Otimização de performance  
✅ **Dashboard Completo**: Monitoramento visual  

### 🔗 Conexão com o Curso:

Este projeto integrou **TODOS** os módulos anteriores:
- **Módulos 1-2**: Fundamentos e Models ✅
- **Módulos 3-4**: Prompts e Chains ✅  
- **Módulo 5**: Memory Systems ✅
- **Módulos 6-8**: RAG Complete ✅
- **Módulos 9-10**: Agents e Projeto 1 ✅

### 🚀 Próximo Módulo (12):
No próximo módulo vamos fazer o **deploy** deste sistema usando **Streamlit**! Vai ficar liiindo! 🌟

**Dica do Pedro**: Guarde este notebook! Ele será a base para nosso app web no próximo módulo!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-11_img_07.png)

## 🎯 Exercício Final: Desafio Completo!

**Seu desafio final**: Criar uma conversa **completa** que use **TODAS** as ferramentas do sistema!

### 📋 Checklist do Desafio:
- [ ] Fazer uma pergunta sobre documentos (RAG)
- [ ] Pedir um cálculo matemático
- [ ] Solicitar uma análise de dados  
- [ ] Buscar informação externa (web)
- [ ] Referenciar conversa anterior (memória)

### 💡 Exemplo de Conversa Completa:
1. "O que é LangChain?"
2. "Quantos componentes principais ele tem ao quadrado?"
3. "Faça uma análise estatística dos dados"
4. "Qual o preço do Bitcoin hoje?"
5. "Baseado em tudo que conversamos, qual seria o ROI de investir em IA?"

**Bora tentar?** 🚀

In [ ]:
# 🎯 SEU DESAFIO FINAL AQUI!
# Crie uma sequência de perguntas que teste todo o sistema

print("🎯 DESAFIO FINAL - Testando TUDO!")
print("=" * 50)

# Lista de perguntas para teste completo
desafio_perguntas = [
    "Explique o que é Machine Learning segundo os documentos",
    "Calcule 15 elevado ao quadrado dividido por 3",
    "Faça uma análise completa dos dados disponíveis",
    "Baseado no que aprendemos sobre ML, qual algoritmo usar para prever vendas?"
]

# Execute o desafio
for i, pergunta in enumerate(desafio_perguntas, 1):
    print(f"\n🔥 PERGUNTA {i}: {pergunta}")
    print("-" * 30)
    
    resposta = agent_master.chat(pergunta)
    print(f"🤖 RESPOSTA: {resposta}")
    print("\n" + "="*50)

print("\n🎉 DESAFIO CONCLUÍDO!")
print("📊 Status final do sistema:")
print(agent_master.get_system_status())